In [ ]:
# TODO Combined with command-line arguments
from config import *

In [ ]:
from pyedflib import EdfReader

class EdfReaderWrapper(EdfReader):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def __enter__(self):
        return self
    def __exit__(self, *args):
        super().close()

In [ ]:
import os
import json
from datetime import timedelta
import pyedflib

outdir = timeline_info_path
data_dir = dataset_path

for indexPatient in range(1, 24+1):
    result_obj = {"record_lst": [], "seizure_lst": [], "unused_rec_idx_lst": []}
    with open(os.path.join(data_dir, f"chb{indexPatient:02d}", f"chb{indexPatient:02d}-summary.txt"), 'r') as f:
        for line in f:
            data=line.split(':')
            if(data[0]=="File Name"):
                edfName=data[1].strip()

                with EdfReaderWrapper(os.path.join(data_dir, f"chb{indexPatient:02d}", edfName)) as pedf: 
                    startTime = pedf.getStartdatetime()
                    startStr = startTime.strftime(dt_fmt)
                    endTime = startTime + timedelta(seconds=pedf.getFileDuration())
                    endStr = endTime.strftime(dt_fmt)
                    result_obj["record_lst"].append({
                        "file": edfName, 
                        "span": [startStr, endStr], 
                        "info": f"{edfName} records {startTime} ~ {endTime} \r\n of shape {pedf.signals_in_file, pedf.getNSamples()[0]}"
                    })
                    
                nextLine = f.readline().strip()
                while (len(nextLine) != 0 and not nextLine.startswith('Number of Seizures in File:')):
                    nextLine = f.readline().strip()
                if nextLine.startswith('Number of Seizures in File:'):
                    for j in range(0, int(nextLine.split(':')[1])):
                        szStartSec = int(f.readline().split(': ')[1].strip().split(' ')[0])
                        szEndSec = int(f.readline().split(': ')[1].strip().split(' ')[0])
                        szStartDT = startTime+timedelta(seconds=szStartSec)
                        szEndDT = startTime+timedelta(seconds=szEndSec)
                        result_obj["seizure_lst"].append({
                            "span": [szStartDT.strftime(dt_fmt), szEndDT.strftime(dt_fmt)], 
                            "info": f'Onset {szStartDT}, last {szEndSec - szStartSec}s'
                        })

        # result_obj["record_lst"].sort(key=lambda obj:obj["span"])
        # result_obj["seizure_lst"].sort(key=lambda obj:obj["span"])
        for k, rec_info in enumerate(result_obj["record_lst"]):
            if rec_info["file"] in ignore_lst:
                result_obj["unused_rec_idx_lst"].append(k)
        #   rec_info["span"] = [rec_info["span"][0].strftime(dt_fmt), rec_info["span"][1].strftime(dt_fmt)]
        
    with open(os.path.join(outdir, f'chb{indexPatient:02d}.json'), 'wt') as f:
        json.dump(result_obj, f, indent=2)